In [72]:
# %pip install -r requirements.txt
# %pip freeze > requirements.txt

INFO: pip is looking at multiple versions of langchain-openai to determine which version is compatible with other requirements. This could take a while.
  Using cached pydantic_settings-2.9.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached langchain_community-0.3.25-py3-none-any.whl.metadata (2.9 kB)
  Using cached zstandard-0.23.0-cp310-cp310-win_amd64.whl.metadata (3.0 kB)
  Using cached urllib3-2.4.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached tenacity-9.1.2-py3-none-any.whl.metadata (1.2 kB)
  Using cached sqlalchemy-2.0.41-cp310-cp310-win_amd64.whl.metadata (9.8 kB)
  Using cached requests_toolbelt-1.0.0-py2.py3-none-any.whl.metadata (14 kB)
INFO: pip is still looking at multiple versions of langchain-openai to determine which version is compatible with other requirements. This could take a while.
  Using cached requests-2.32.4-py3-none-any.whl.metadata (4.9 kB)
  Using cached pydantic-2.11.5-py3-none-any.whl.metadata (67 kB)
  Using cached packaging-24.2-py3-none-any.whl.

ERROR: Cannot install langchain-openai==0.3.22 and openai==0.28.0 because these package versions have conflicting dependencies.
ERROR: ResolutionImpossible: for help visit https://pip.pypa.io/en/latest/topics/dependency-resolution/#dealing-with-dependency-conflicts


In [ ]:
from jinja2 import Template
import openai

from dotenv import load_dotenv
import json
import os
import random

In [76]:
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

In [77]:
JSON_FILE_PATH = "data\\project_summary.json"
RCMD_PROMPT_PATH = "cat-feat_recommend.md"

In [78]:
# Read data
with open(JSON_FILE_PATH, 'r', encoding='utf-8') as f:
    json_data = json.load(f)

# Read prompt
with open(RCMD_PROMPT_PATH, "r", encoding="utf-8", errors="replace") as f:
    RCMD_PROMPT = f.read()

In [79]:
json_data

[{'project_info_id': 1,
  'category': '웹 서비스',
  'core_features': ['ERD 자동생성', 'API 명세서 자동생성', '프로젝트 정보 자동생성'],
  'created_at': '2025-06-18T00:39:46.947095',
  'problem_solving': {'solutionIdea': 'AI가 자동으로 설계 문서를 생성하여 개발 과정을 단순화',
   'currentProblem': '초보 개발자들이 복잡한 설계 과정을 이해하고 진행하는 데 어려움이 있음',
   'expectedBenefits': ['빠른 개발 착수', '효율적인 팀원 간 커뮤니케이션', '체계적인 프로젝트 경험']},
  'target_users': ['프로젝트 경험이 적은 초보 개발자'],
  'technology_stack': ['react',
   'typescript',
   'springboot',
   'AWS',
   'git action',
   'vercel'],
  'title': '프로젝트 관리 웹 서비스',
  'updated_at': '2025-06-18T00:39:46.947102',
  'workspace_id': 1},
 {'project_info_id': 11,
  'category': '학습 관리',
  'core_features': ['체크리스트 형태의 과제 관리', '퀴즈를 통한 반복 학습'],
  'created_at': '2025-06-19T08:24:29.404944',
  'problem_solving': {'solutionIdea': '스터디 그룹 내에서 과제 관리와 퀴즈를 통해 학습 내용을 반복 학습할 수 있는 기능 제공',
   'currentProblem': '학생들이 학습 동기를 잃고 효과적으로 학습하지 못하는 문제',
   'expectedBenefits': ['학습 효과 극대화', '사용자 간 소통 강화', '학습 동기 부여']},
  'target_users': ['학습

In [80]:
print(RCMD_PROMPT)

당신은 신입/초보 개발자 팀을 이끄는 시니어 테크 리드입니다.  
다음은 한 프로젝트에 대한 주요 정보입니다.  
이 정보를 바탕으로 해당 프로젝트를 성공적으로 구현하기 위한 작업 구조를 작성해야 합니다.

---

- 프로젝트 정보 JSON:
{{ input }}

# 작업 조건:
1. Category: 패키지 수준의 상위 기능 그룹, 5개 이상  
2. Feature: 각 Category 아래 기능 모듈, 5개 이상  
3. Action: 각 Feature에 대응하는 실제 구현 단위, 3개 이상  
4. importance: 각 Actions의 기능 중요도 (1~5 사이 정수)  
5. 출력은 반드시 JSON 형식의 문자열만 반환 (설명 X, 주석 X)

---

아래의 **조건을 반드시 모두 충족한 작업 구조**를 작성하십시오.  
하나라도 누락될 경우, 출력은 무효입니다. 반드시 모든 조건을 만족해야 합니다:

1. **Category**는 5개 이상이어야 합니다.  
2. **각 Category마다 Feature는 무조건 5개 이상이어야 합니다. 예외는 없습니다.**  
3. **각 Feature에는 Action이 3개 이상 포함되어야 합니다.**  
4. **모든 Action에는 중요도를 나타내는 `importance` 값(1~5 정수)이 있어야 합니다.**
5. **반드시 JSON 형식만 출력하십시오. 주석, 설명, 텍스트, 문장 절대 금지입니다.**


## Action 작성 시 반드시 지켜야 할 추가 조건:

- 각 Action은 **UI 수준의 일반 설명이 아니라, 구체적인 기술적 구현 단위**로 작성해야 합니다.
- 프로젝트에 명시된 기술 스택 (예: `react`, `typescript`, `springboot`, `langchain`, `컴퓨터 비전`)을 **적극적으로 반영**하십시오.
- 예시:
  - `"LangChain을 이용한 문맥 기반 응답 처리"`
  - `"SpringBoot로 작성된 API에 JWT 인증 로직 추가"`
  

In [81]:
# project ID 추출
def extract_values_by_key(obj, key):
    values = []

    if isinstance(obj, dict):
        for k, v in obj.items():
            if k == key:
                values.append(v)
            values.extend(extract_values_by_key(v, key))
    elif isinstance(obj, list):
        for item in obj:
            values.extend(extract_values_by_key(item, key))

    return values

# Read target file
with open(JSON_FILE_PATH, "r", encoding="utf-8") as f:
    data = json.load(f)

target_key = "project_info_id"
proj_id_list = extract_values_by_key(data, target_key)

print(proj_id_list)

[1, 11, 4, 12, 6, 16, 15]


In [82]:
project_id = int(random.choice(proj_id_list))

def extract_project_summary(input_json, id):
    # peject_id로 프로젝트 개요 중 지정 컬럼 읽어오기
    for item in input_json:
        if item.get("project_info_id") == id:
            return {
                "category": item["category"],
                "core_features": item["core_features"],
                "problem_solving": {
                    "solutionIdea": item["problem_solving"]["solutionIdea"],
                    "currentProblem": item["problem_solving"]["currentProblem"],
                    "expectedBenefits": item["problem_solving"]["expectedBenefits"]
                },
                "target_users": item["target_users"],
                "technology_stack": item["technology_stack"],
                "title": item["title"],
                "workspace_id": item["workspace_id"]
            }
        
    return None

# peject_id로 읽어온 프로젝트 개요의 지정 컬럼
project_summary = extract_project_summary(data, project_id)
# project_summary  # test
type(project_summary)  # dict

if project_summary:
    print(json.dumps(project_summary, ensure_ascii=False, indent=2))  # <class 'str'>
else:
    print("** 해당 project_id를 찾을 수 없습니다. **")

{
  "category": "웹 서비스",
  "core_features": [
    "ERD 자동생성",
    "API 명세서 자동생성",
    "프로젝트 정보 자동생성"
  ],
  "problem_solving": {
    "solutionIdea": "AI가 자동으로 설계 문서를 생성하여 개발 과정을 단순화",
    "currentProblem": "초보 개발자들이 복잡한 설계 과정을 이해하고 진행하는 데 어려움이 있음",
    "expectedBenefits": [
      "빠른 개발 착수",
      "효율적인 팀원 간 커뮤니케이션",
      "체계적인 프로젝트 경험"
    ]
  },
  "target_users": [
    "프로젝트 경험이 적은 초보 개발자"
  ],
  "technology_stack": [
    "react",
    "typescript",
    "springboot",
    "AWS",
    "git action",
    "vercel"
  ],
  "title": "프로젝트 관리 웹 서비스",
  "workspace_id": 1
}


In [83]:
# test
project_summary
# type(project_summary)  # dict

{'category': '웹 서비스',
 'core_features': ['ERD 자동생성', 'API 명세서 자동생성', '프로젝트 정보 자동생성'],
 'problem_solving': {'solutionIdea': 'AI가 자동으로 설계 문서를 생성하여 개발 과정을 단순화',
  'currentProblem': '초보 개발자들이 복잡한 설계 과정을 이해하고 진행하는 데 어려움이 있음',
  'expectedBenefits': ['빠른 개발 착수', '효율적인 팀원 간 커뮤니케이션', '체계적인 프로젝트 경험']},
 'target_users': ['프로젝트 경험이 적은 초보 개발자'],
 'technology_stack': ['react',
  'typescript',
  'springboot',
  'AWS',
  'git action',
  'vercel'],
 'title': '프로젝트 관리 웹 서비스',
 'workspace_id': 1}

In [84]:
# Jinja2 template 객체 생성
template = Template(RCMD_PROMPT)

# Rendering
rendered = template.render(input=json.dumps(project_summary, ensure_ascii=False, indent=2))
print(rendered)

당신은 신입/초보 개발자 팀을 이끄는 시니어 테크 리드입니다.  
다음은 한 프로젝트에 대한 주요 정보입니다.  
이 정보를 바탕으로 해당 프로젝트를 성공적으로 구현하기 위한 작업 구조를 작성해야 합니다.

---

- 프로젝트 정보 JSON:
{
  "category": "웹 서비스",
  "core_features": [
    "ERD 자동생성",
    "API 명세서 자동생성",
    "프로젝트 정보 자동생성"
  ],
  "problem_solving": {
    "solutionIdea": "AI가 자동으로 설계 문서를 생성하여 개발 과정을 단순화",
    "currentProblem": "초보 개발자들이 복잡한 설계 과정을 이해하고 진행하는 데 어려움이 있음",
    "expectedBenefits": [
      "빠른 개발 착수",
      "효율적인 팀원 간 커뮤니케이션",
      "체계적인 프로젝트 경험"
    ]
  },
  "target_users": [
    "프로젝트 경험이 적은 초보 개발자"
  ],
  "technology_stack": [
    "react",
    "typescript",
    "springboot",
    "AWS",
    "git action",
    "vercel"
  ],
  "title": "프로젝트 관리 웹 서비스",
  "workspace_id": 1
}

# 작업 조건:
1. Category: 패키지 수준의 상위 기능 그룹, 5개 이상  
2. Feature: 각 Category 아래 기능 모듈, 5개 이상  
3. Action: 각 Feature에 대응하는 실제 구현 단위, 3개 이상  
4. importance: 각 Actions의 기능 중요도 (1~5 사이 정수)  
5. 출력은 반드시 JSON 형식의 문자열만 반환 (설명 X, 주석 X)

---

아래의 **조건을 반드시 모두 충족한 작업 구조**를 작성하십시오.  
하나라도 누락될 경우, 출력은 무효입니다.

In [ ]:
response = openai.ChatCompletion.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "당신은 신입/초보 개발자 팀을 이끄는 시니어 테크 리드입니다. 주어진 정보를 바탕으로 해당 프로젝트를 성공적으로 구현하기 위한 작업 구조를 작성해야 합니다."},
        {"role": "user", "content": rendered}
    ],
    temperature=0.3
)

In [86]:
generated = response.choices[0].message.content.strip()
print(generated)

{
  "workspace_id": 1,
  "recommendedCategories": [
    {
      "name": "설계 문서 자동화",
      "features": [
        {
          "name": "ERD 자동생성",
          "actions": [
            { "name": "TypeScript로 ERD 생성 로직 구현", "importance": 5 },
            { "name": "React에서 ERD 시각화 컴포넌트 개발", "importance": 4 },
            { "name": "SpringBoot로 ERD 데이터 저장 API 구현", "importance": 5 }
          ]
        },
        {
          "name": "API 명세서 자동생성",
          "actions": [
            { "name": "OpenAPI 스펙을 기반으로 API 문서 생성 로직 구현", "importance": 5 },
            { "name": "React에서 API 명세서 표시 컴포넌트 개발", "importance": 4 },
            { "name": "SpringBoot에서 Swagger 통합 설정", "importance": 5 }
          ]
        },
        {
          "name": "프로젝트 정보 자동생성",
          "actions": [
            { "name": "TypeScript로 프로젝트 정보 수집 모듈 개발", "importance": 4 },
            { "name": "React에서 프로젝트 정보 표시 UI 구현", "importance": 4 },
            { "name": "SpringBoot에서 프로젝트 정보 저장 API 구현", "importance": 5 }
        